In [1]:
import logging
import json
import os
import requests
import re
import time
from datetime import datetime
import pandas as pd
import numpy as np
from getpass import getpass
#from progress.bar import IncrementalBar
from tqdm import tqdm
import numba
from numba import jit


# SPecial function for token retrieving, or saving in case it's valid
# Logic for token retrievement should be as follows:
# User input: Login and Password
# Also for different API its necessary to provide endpoint, however I think 
# that token should be valid for all services. NEED  TO CHECK.


# Endpoint for SAM reports: https://repcenter.skf.com/rest2/sam_reports/api/Authentication
# Endpoint for Measurements: https://repcenter.skf.com/rest/mhv/api/v1/Authorization/login
#logging.basicConfig(filename = 'tokenlog.log',
#                    format = '%(asctime)s:%(levelname)s -%(name)s - %(message)s', 
#                    level = logging.INFO)

# Script can be used for Authentification on both
def retrieveTokenAnalyst(username = 'user1',
                         password = 'password',
                         API = 'measurements',
                         logger_name = ''): 
    # Setting proper logger
    logger = logging.getLogger(logger_name)
    
    # Definition of the endpoints for Anlyst API
    if API == 'SAM':
        endpoint = 'https://repcenter.skf.com/rest2/sam_reports/api/Authentication'
    elif API == 'measurements':
        endpoint = 'https://repcenter.skf.com/rest/mhv/api/v1/Authorization/login'
    else:
        print('Select proper API. possible options are: SAM or measurements')
        logger.warning(f'No valid API selected. Was selected {API} while proper options are: SAM or measurements')
    
    # If user didn't provide password in call to function we are asking to provide
    if password == 'password':
        password = getpass()
    
    # Necessary details for POST request
    auth_details = {'username': username, 'password': password}
    header = {'accept': 'text/plain', 'Content-Type': 'application/json'}
    
    # Request
    auth_req = requests.post(endpoint, headers = header, data = json.dumps(auth_details))
    
    # Recieved results in case of successful or unsuccessfull request
    if auth_req.status_code == 200:
        # We need to document succesfull attemt in logger
        # we need to retrieve information about token and expiration date
        token = auth_req.json()['token']
        exp_date = auth_req.json()['tokenExpireAt']
        logger.info(f'Token recieved successfully. Expiration date: {exp_date}')
        return {'token': token, 'exp_date': exp_date}
    else:
        status = auth_req.status_code
        logger.warning(f'Token has NOT recieved. Status of request: {status}. Response: {auth_req.content}')
        response = json.dumps(auth_req.json())
        
        return {'token': None, 'exp_date': None}

def getListOfPointsAnalyst(customer = 'customer', 
                           token = {},
                           username = 'user1',
                           password = 'password',
                           pageSize = 5000, 
                           pageNumber = 1,
                           logger_name = ''):
    # Setting proper logger
    logger = logging.getLogger(logger_name)
    
    logger.info('Getting ANALYST list of points')
    
    # Checking for input variables.
    if customer == 'customer':
        logger.warning('No inforation about customer provided. Unable to retrieve information from ANALYST without customer name')
        return None
    if bool(token) == False and (password == 'password' or username == 'user1'):
        logger.warning(f'It\'s necessary to provide token or credentials for token retrievement. No information about credentialsand/or token is provided')
        return None
    
    # Retrieving token if it wasn't provided
    if bool(token) == False:
        logger.info('No token was provided. Trying to retrieve token from Auth service.')
        token = retrieveTokenAnalyst(username = username, password = password)
    if token == None:
        logger.error('Token retrieving was unsuccessful. Impossible to retrieve list of points from ANALYST')
        return None
        
    endpoint = f'https://repcenter.skf.com/rest/mhv/api/v1/{customer}/Points/details?pageSize={pageSize}&pageNumber={pageNumber}'
    auth_token = f'Bearer {token["token"]}'
    header = {'accept': 'text/plain', 'Authorization': auth_token}
    
    list_of_points_req = requests.get(endpoint, headers = header)
    
    
    if list_of_points_req.status_code == 200:
        list_of_points = pd.DataFrame(list_of_points_req.json()['results'])
        list_of_points['presented_in_measurement_points'] = True
        if list_of_points_req.json()['pageCount'] > 1:
            logger.info(f"API answer contains more than one page. Will request for {list_of_points_req.json()['pageCount'] - 1} additional pages.")
            for page in range(2, list_of_points_req.json()['pageCount'] + 1):
                pageNumber = page
                endpoint = f'https://repcenter.skf.com/rest/mhv/api/v1/{customer}/Points/details?pageSize={pageSize}&pageNumber={pageNumber}'
                list_of_points_req = requests.get(endpoint, headers = header)
                list_of_points = pd.concat([list_of_points, pd.DataFrame(list_of_points_req.json()['results'])])
                logger.info(f'Page {pageNumber} was received')
        
        return list_of_points
    else:
        logger.error(f'Something went wrong. Request status: {list_of_points_req.status_code}.')
        return None
    

In [7]:
tmp = getListOfPointsAnalyst(customer='BRAEO3718', username='integration.enlight', password='Brazil2021')
unique_list = list(tmp.pointName.unique())

ConnectionError: HTTPSConnectionPool(host='repcenter.skf.com', port=443): Max retries exceeded with url: /rest/mhv/api/v1/Authorization/login (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000015ABEB72E20>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

In [5]:
# The best option will be to feed list of unique values to this function
def check_names(mp_names = [], logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    #Validation of input
    if not isinstance(mp_names, list):
        log.error(f'Check names recieved wrong data structure. Accepted structure: list')
        return None
    
    # Checking for regular vibration points
    regex_vibr = '^((MA)|(MI)|(ME)|(OS)|(TO)|(DV)|(OI))?( |^)\d{2}(A|H|V|R)(A|T|V|S|B|P|G|D|(E1)|(E2)|(E3)|(E4)) ?\w*? ?((DE)|(NDE))? ?(\w{1,})?$'
    r = re.compile(regex_vibr)
    good_list = list(filter(r.match, mp_names))
    log.info(f'Among {len(mp_names)} unique names, {len(good_list)} names with good patter for vibration points.')
    first_rejected = list(set(mp_names) - set(good_list))
    log.info(f'{len(first_rejected)} unique names have patetrn that are different from vibration points')
    
    # Checking for MI SIT
    regex_misit = 'M(I|A) SIT'
    r_sit = re.compile(regex_misit)
    misit_list = list(filter(r_sit.match, first_rejected))
    log.info(f'Among {len(first_rejected)} unique names, {len(misit_list)} names with paterrns MI SIT/MA SIT')
    second_rejected = list(set(first_rejected) - set(misit_list))
    log.info(f'{len(second_rejected)} unique names has pattern that are not vibrationa and not MI|A SIT')
    
    #Checking for points that are 01S Manual entry REP
    regex_manentry = '[0-9]{2}S [Mm]anual [Ee]ntry'
    r_manentry = re.compile(regex_manentry)
    manentry_list = list(filter(r_manentry.match, second_rejected))
    log.info(f'Among {len(second_rejected)} unique names, {len(manentry_list)} names with paterrns Manual Entry')
    third_rejected = list(set(second_rejected) - set(manentry_list))
    log.info(f'{len(third_rejected)} unique names has pattern that are not vibrationa and not MI|A SIT and not Manual entry points')

    #Crating a list of good names
    good_names = list(set(mp_names) - set(third_rejected))
    
    return {'good_names': good_names, 'wrong_names': third_rejected}
    
# Function for checking for problems in rejected names. Function should check for few main problems with the names
#1. Check for wrong first two letters. Wrong Device
#2. Lack or excessive digits in the begining
#3. Wrong orientation
#4. Wrong Measurement type
#5. Extra spaces between number/orientation and Measurement type? Not sure how to identify! :(
#6. Unknown problems
# The result of the function should be dictionary with identified problems


def define_names_problems(wrong_names = [],
                          logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    #Validation of input
    if not isinstance(wrong_names, list):
        log.error(f'Define problems recieved wrong data structure. Accepted structure: list')
        return None
    if len(wrong_names) == 0:
        log.warning('There is no wrong names provided. Skipping problems detection.')
        return None
    
    names_problems = {}
    log.info(f'Creating a dictionary for results storage. Following MP names provided for analysis {", ".join(wrong_names)}')
    for name in wrong_names:
        names_problems[name] = []
    
    #Checking for wrong two letters
    regex_device = '^(?:(?=\w{2})((MA)|(MI)|(ME)|(OS)|(TO)|(DV)|(OI)))|((?=\d)\d)'
    r_device = re.compile(regex_device)
    good_device = list(filter(r_device.match, wrong_names))
    log.info(f'{len(good_device)} have proper or no device identification in the name. Follow')
    wrong_device = list(set(wrong_names) - set(good_device))
    
    if len(wrong_device) > 0:
        log.warning(f'{len(wrong_device)} points have name with wrong device identification: {", ".join(wrong_device)}')
        for name in wrong_device:
            names_problems[name].append('Device identification is wrong')
    
    # Checking for lack or excessive numbers in name
    regex_numbers = '(^\w*)?( |^)\d{2}([a-zA-Z]| )'
    r_numbers = re.compile(regex_numbers)
    good_numbers = list(filter(r_numbers.match, wrong_names))
    log.info(f'{good_numbers} points has names with proper bearing number.')
    wrong_numbers = list(set(wrong_names) - set(good_numbers))
    
    if len(wrong_numbers) > 0:
        log.warning(f'{len(wrong_numbers)} points have name with wrong bearing number. Bearing number should have two digits: {", ".join(wrong_numbers)}')
        for name in wrong_numbers:
            names_problems[name].append('Bearing number is wrong')
            
    # Checking for correct orientation
    regex_orient = '(^\w*)?( |^)\d*(A|H|V|R)'
    r_orient = re.compile(regex_orient)
    good_orient = list(filter(r_orient.match, wrong_names))
    log.info(f'{len(good_orient)} points has good orientation notation.')
    wrong_orient = list(set(wrong_names) - set(good_orient))
    
    if len(wrong_orient) > 0:
        log.warning(f'{len(wrong_orient)} points have name with wrong orientation notation: {", ".join(wrong_orient)}')
        for name in wrong_orient:
            names_problems[name].append('Orientation notation is wrong')
    
    # Checking for correct measurement type
    regex_meastype = '(^\w*)?( |^)\d*.*(A|T|V|S|B|P|G|D|(E1)|(E2)|(E3)|(E4))( |$)'
    r_meastype = re.compile(regex_meastype)
    good_meastype = list(filter(r_meastype.match, wrong_names))
    log.info(f'{len(good_meastype)} have proper measurement type notation')
    wrong_meastype = list(set(wrong_names) - set(good_meastype))
    
    if len(wrong_meastype) > 0:
        log.warning(f'{len(wrong_meastype)} points have name with wrong measurement type: {", ".join(wrong_meastype)}')
        for name in wrong_meastype:
            names_problems[name].append('Measurement type is wrong')
    
    # Unknown problems. All the points for which the problem wasn't identified 
    # to unknown problems. All the points which have more than one problem has
    # Unknown or multiple problems
    for name in names_problems:
        if len(names_problems[name]) != 1:
            log.warning(f'Point {name}  has multiple or unknon problems.')
            names_problems[name] = f'Unidentified or multiple problems. Possible suggestions: {", ".join(names_problems[name]) }'
        if len(names_problems[name]) == 1:
            names_problems[name] = names_problems[name][0]
    return {'names_issues': names_problems}


def prepare_hierarchy(treelem = pd.DataFrame(),
                      logger = ''):
    
    # Setting logger
    log = logging.getLogger(logger)
    
    #Validationof the input
    if not validate_treelems(treelem, logger):
        return None
    
    #Creating path for easier identification
    levels = list(treelem.BRANCHLEVEL.unique())
    levels.sort()
    log.info(f'Creating PATH for each element of tree. Hierary in total has following unique levels: {levels}')
    treelem['PATH'] = ""
    for level in levels:
        if level == 0:
            continue
        print(level)
        tmp_df = treelem[(treelem.BRANCHLEVEL == level)]
        log.info(f'Creating hierarchy for {level} level. Level has {len(tmp_df)} elements. Element names are {tmp_df["NAME"]}')
        for element in tmp_df.TREEELEMID.unique():
            print(element)
            print(treelem[treelem.TREEELEMID == element])
            parent_id = treelem.loc[treelem.TREEELEMID == element, 'PARENTID'].item()
            parent_path = treelem.loc[treelem.TREEELEMID == parent_id, 'PATH'].item()
            element_name = treelem.loc[treelem.TREEELEMID == element, 'NAME'].item()
            treelem.loc[treelem.TREEELEMID == element, 'PATH'] = parent_path + "/" + element_name
    
    #Creating list of  measurement point, assets and fls
    mps = treelem.loc[treelem.CONTAINERTYPE == 4, ['NAME', 'TREEELEMID']]
    assets = treelem.loc[treelem.CONTAINERTYPE == 3, ['NAME', 'TREEELEMID']]
    fls = treelem.loc[(treelem.CONTAINERTYPE == 2) & (treelem.BRANCHLEVEL >= (max(levels) - 2)), ['NAME', 'TREEELEMID']]
    log.info(f'Hierarchy has {len(fls)} Functional locations, {len(assets)} assets and {len(mps)} measurement points')
    
    #Creating clear identification for disabled points
    for fl in fls.TREEELEMID:
        if treelem.loc[treelem.TREEELEMID == fl, 'ELEMENTENABLE'].item() == 0:
            log.warning(f'FL with ID {fl} completely disabled. All assets and mp indide it will be marked and counted as disabled.')
            dis_assets = list(treelem.loc[treelem.PARENTID == fl, 'TREEELEMID'])
            treelem.loc[treelem.TREEELEMID.isin(dis_assets), 'ELEMENTENABLE'] = 0
    for asset in assets.TREEELEMID:
        if treelem.loc[treelem.TREEELEMID == asset, 'ELEMENTENABLE'].item() == 0:
            log.warning(f'Asset with ID {asset} completely disabled all mp inside will be marked and counted as disabled.')
            treelem.loc[treelem.PARENTID == asset, 'ELEMENTENABLE'] = 0
            
    results = {'mp_names': mps, 'assets_names': assets, 'fl_names': fls, 'hierarchy_depth': max(levels), 'treeelem': treelem}
    
    return results

def check_sit(treelem = pd.DataFrame(),
              logger = ''):
    
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the imput.
    if not validate_treelems(treelem, logger):
        return None
    
    # Checking for MI SIT point in each FL if we have at least half of the FL with them
    mp_names = list(treelem.loc[treelem.CONTAINERTYPE == 4, 'NAME'])
    regex_misit = 'M(I|A) SIT'
    r_sit = re.compile(regex_misit)
    misit_list = list(filter(r_sit.match, mp_names))
    
    # Creating list of MPs for each functional location
    #List of FL
    mask_fl = (treelem.CONTAINERTYPE == 2) & (treelem.BRANCHLEVEL >= (max(treelem.BRANCHLEVEL) - 2))
    fls = treelem.loc[mask_fl, ['NAME', 'TREEELEMID']]
    mp_in_fl = {}
    for fl_id in list(fls['TREEELEMID']):
        mp_in_fl[fl_id] = []
    for fl_id in list(fls['TREEELEMID']):
        asset_ids = list(treelem.loc[treelem.PARENTID == fl_id, 'TREEELEMID'])
        for asset_id in asset_ids:
            mp = list(treelem.loc[treelem.PARENTID == asset_id, 'NAME'])
            mp_in_fl[fl_id] = mp_in_fl[fl_id] + mp
        log.info(f'FL with id {fl_id} has {len(mp_in_fl[fl_id])} measurement points with following names: {mp_in_fl[fl_id]}')
    
    # Checking the rule that if customer uses MI SIT than each maesurement location 
    # should have at least one and it should be located in Motor component! 
    log.info(f'Checking hierachy for SIT points potential problems')
    sit_problems = {'missing_sit': [], 'excessive_sit': [], 'good_sit': []}
    if len(misit_list) > len(fls)/2:
        for fl_id in mp_in_fl.keys():
            misit_in_fl = list(filter(r_sit.match, mp_in_fl[fl_id]))
            if len(misit_in_fl) == 0:
                log.warning(f'FL with ID {fl_id} has no SIT point in any asset. Need to add')
                sit_problems['missing_sit'].append(fl_id)
            if len(misit_in_fl) > 1:
                log.warning(f'FL with ID {fl_id} has more than one SIT points. Need to remove excessive points.')
                sit_problems['excessive_sit'].append(fl_id)
            if len(misit_in_fl) == 1:
                log.info(f'FL with ID {fl_id} has SIT point.')
                sit_problems['good_sit'].append(fl_id)
                
    # Checking the problem that MI SIT points should be presented in Motor component
    # list of assets:
    mask_asset = treelem.CONTAINERTYPE == 3
    assets = treelem.loc[mask_asset, ['NAME', 'TREEELEMID', 'FilterKey']]
    #Checking for assets without Filter Key Assigned
    assets_wo_filterkey = assets.loc[assets.FilterKey.isna(), ['NAME', 'TREEELEMID']]
    if len(assets_wo_filterkey) > 0:
        log.warning(f'There are {len(assets_wo_filterkey)} assets without defined filter key. The names are: {list(assets_wo_filterkey.NAME)}')
    else:
        log.info('All assets have assigned filter key.')
        
    assets_motors = assets.loc[assets.FilterKey == '*Motor', ['NAME', 'TREEELEMID', 'FilterKey']]
    assets_other = assets.loc[assets.FilterKey != '*Motor', ['NAME', 'TREEELEMID', 'FilterKey']]
    
    #Checking that all Motor Assets has MI SIT
    sit_problems['motors_wo_SIT'] = []
    sit_problems['duplicated_SIT_in_motor'] = []
    sit_problems['other_components_w_SIT'] = []
    
    if len(misit_list) > len(fls)/2:
        for motor in assets_motors.TREEELEMID:
            mp_asset = treelem.loc[treelem.PARENTID == motor, 'NAME']
            misit_in_asset = list(filter(r_sit.match, mp_asset))
            if len(misit_in_asset) == 0:
                log.warning(f'Asset with ID {motor} and filter Key Motor has no SIT point in any asset. Need to add')
                sit_problems['motors_wo_SIT'].append(motor)
            if len(misit_in_asset) > 1:
                log.warning(f'Asset with ID {motor} and Filter Key has more than one SIT points. Need to remove excessive points.')
                sit_problems['duplicated_SIT_in_motor'].append(motor)
            if len(misit_in_asset) == 1:
                log.info(f'Asset with ID {motor} and Filter Key Motor has SIT point.')
        for component in assets_other.TREEELEMID:
            mp_asset = treelem.loc[treelem.PARENTID == component, 'NAME']
            misit_in_asset = list(filter(r_sit.match, mp_asset))
            if len(misit_in_asset) == 0:
                log.info(f'Asset with ID {component} and filter key {treelem.loc[treelem.TREEELEMID == component, "FilterKey"].item()} has no MI SIT points')
            if len(misit_in_asset) >= 1:
                log.warning(f'Asset with ID {component} and Filter Key {treelem.loc[treelem.TREEELEMID == component, "FilterKey"].item()} has SIT points.')
                sit_problems['other_components_w_SIT'].append(component)

    
    return {'sit_issues': sit_problems}


                
        
        
def check_hierarchy(treelem = pd.DataFrame(), 
                    logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the imput.
    if not validate_treelems(treelem, logger):
        return None
    
    # Checking for MI SIT point in each FL if we have at least half of the FL with them
    mp_names = list(mps['NAME'])
    regex_misit = 'M(I|A) SIT'
    r_sit = re.compile(regex_misit)
    misit_list = list(filter(r_sit.match, mp_names))
    
    # Creating list of MPs for each functional location
    
    # Checking the rule that if customer uses MI SIT than each maesurement location 
    # should have at least one and it should be located in Motor component! 
    # NEED ADDITIONAL DATA TO CHECK THIS
    log.info(f'Checking hierachy for SIT points potential problems')
    sit_problems = {'missing_sit': [], 'excessive_sit': [], 'good_sit': []}
    if len(misit_list) > len(fls)/2:
        for fl_id in mp_in_fl.keys():
            misit_in_fl = list(filter(r_sit.match, mp_in_fl[fl_id]))
            if len(misit_in_fl) == 0:
                log.warning(f'FL with ID {fl_id} has no SIT point in any asset. Need to add')
                sit_problems['missing_sit'].append(fl_id)
            if len(misit_in_fl) > 1:
                log.warning(f'FL with ID {fl_id} has more than one SIT points. Need to remove excessive points.')
                sit_problems['excessive_sit'].append(fl_id)
            if len(misit_in_fl) == 1:
                log.info(f'FL with ID {fl_id} has SIT point.')
                sit_problems['good_sit'].append(fl_id)
    
    #Duplications problems
    log.info(f'Checking hierarchy for ducplicated names in the same FL')
    duplication_problems = {}
    for fl_id in mp_in_fl.keys():
        duplication_problems[fl_id] = []
    for fl_id in mp_in_fl.keys():
        for name in set(mp_in_fl[fl_id]):
            name_count = mp_in_fl[fl_id].count(name)
            if name_count > 1:
                duplication_problems[fl_id].append(name)
        if len(duplication_problems[fl_id]) > 0:
            log.warning(f'Fucntional location {fl_id} contains points with duplicated names: List of duplicated names are: {", ".join(duplication_problems[fl_id])}')
    
    log.info(f'Checking for sequence of measurement points in functional locations')
    
    
    #Checking for too short hierachy
    
    
    
    result = {'mp_in_fl': mp_in_fl, 
              'sit_stat': sit_problems, 
              'duplications_stat': duplication_problems, 
              'sequence_stat': sequence_problems}
    
    
    
    
    return result

def validate_treelems(tree_df = pd.DataFrame(), 
                      logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    validated = True
    if not isinstance(tree_df, pd.DataFrame):
        log.warning(f'Provided input has incorrect type. Required type: pd.DatFrame, received input: {type(tree_df)}')
        return None
    
    required_columns = ['TREEELEMID', 'PARENTID', 'CONTAINERTYPE', 'NAME', 
                        'ELEMENTENABLE', 'PARENTENABLE', 'ChannelEnable', 'HIERARCHYTYPE',
                        'TBLSETID', 'BRANCHLEVEL', 'PointUnitType', 'FilterEnvelope',
                        'PointSensorUnitType', 'PointOrientation', 'PointLocation', 'DADType',
                        'FilterKey', 'SCALARALRMID', 'ALARMMETHOD', 'DANGERHI', 'DANGERLO',
                        'ALERTHI', 'ALERTLO', 'ENABLEALERTHI', 'ENABLEALERTLO',
                        'ENABLEDANGERHI', 'ENABLEDANGERLO']
    if set(required_columns).issubset(tree_df.columns):
        log.info(f'Provided dataframe has all necessary information for the analysis: {tree_df.columns}')
    else:
        missing_columns = list(set(required_columns) - set(tree_df.columns))
        log.warning(f'Provided dataframe contains not all information needed. Check the input. Missing columns: {missing_columns}')
        validated = False
    
    if len(tree_df) == 0:
        log.warning(f'Provided dataframe is empty.')
        validated = False
    
    return validated
   



            
    

def check_location(treelem = pd.DataFrame(), 
                    logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the input
    if not validate_treelems(treelem, logger):
        return None
    
    tmp_df = treelem[(treelem.CONTAINERTYPE == 4) & (~treelem.NAME.isin(['MI SIT', 'MA SIT']))]
    tmp_df.reset_index(drop = True, inplace = True)
    #treelem['PointLocation'] = [np.NaN if pd.isnull(x) else x for x in treelem['PointLocation']]
    
    locations_name = []
    for name in tmp_df['NAME']:
        try:
            number = re.search('[1-9]{1,3}', name).group(0)
            locations_name.append(number)
        except AttributeError:
            locations_name.append(np.NaN)
    
    locations_set = list(tmp_df['PointLocation'])
    
    diff = [lset != lname for lset, lname in zip(locations_set, locations_name)]
    tmp_df['Location'] = locations_set
    results_df = tmp_df.loc[diff, ['TREEELEMID', 'NAME', 'Location', 'Path']]
    #results_df['Path'] = [create_path(node_id = x, treelem = treelem) for x in results_df.TREEELEMID]

    return results_df.to_dict()
        
    
def check_orientation(treelem = pd.DataFrame(),
                      logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the input
    if not validate_treelems(treelem, logger):
        return None
    
    tmp_df = treelem[(treelem.CONTAINERTYPE == 4) & (~treelem.NAME.isin(['MI SIT', 'MA SIT']))]
    tmp_df.reset_index(drop = True, inplace = True)
    #treelem['PointLocation'] = [np.NaN if pd.isnull(x) else x for x in treelem['PointLocation']]
    
    orientations_name = []
    orientation_mapping = {'H': 'Horizontal', 'V': 'Vertical', 'A': 'Axial', 'R': 'Radial'}
    for name in tmp_df['NAME']:
            try:
                orientation = re.search('(^\w{2})?( |^)[0-9]{1,3}(A|H|V|R)', name).group(0)
                orientations_name.append(orientation_mapping[orientation[-1]])
            except AttributeError:
                orientations_name.append(np.NaN)
    orientations_set = list(tmp_df['PointOrientation'])
    
    diff = [oset != oname for oset, oname in zip(orientations_set, orientations_name)]
    
    tmp_df['Orientation'] = orientations_set
    results_df = tmp_df.loc[diff, ['TREEELEMID', 'NAME', 'Orientation', 'Path']]
    #results_df['Path'] = [create_path(node_id = x, treelem = treelem) for x in results_df.TREEELEMID]

    return results_df.to_dict()
    
def db_stat(treelem = pd.DataFrame(), 
            logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the input
    if not validate_treelems(treelem, logger):
        return None
    
    #Creating statistics
    #General statistics
    n_mp = len(treelem[treelem.CONTAINERTYPE == 4])
    n_assets = len(treelem[treelem.CONTAINERTYPE == 3])
    n_fl = len(treelem[(treelem.CONTAINERTYPE == 2) & (treelem.BRANCHLEVEL == max(treelem.BRANCHLEVEL) - 2)])
    
    #Disabled Statistics
    n_dis_mp = len(treelem[(treelem.CONTAINERTYPE == 4) & (treelem.ELEMENTENABLE == 0)])
    n_dis_mp_perc = round(n_dis_mp/n_mp*100, 2)
    
    n_dis_assets = len(treelem[(treelem.CONTAINERTYPE == 3)&(treelem.ELEMENTENABLE == 0)])
    n_dis_assets_perc = round(n_dis_assets/n_assets*100, 2)
    
    mask_disfl = (treelem.CONTAINERTYPE == 4)&(treelem.BRANCHLEVEL == max(treelem.BRANCHLEVEL) - 2)&(treelem.ELEMENTENABLE == 0) 
    n_dis_fl = len(treelem[mask_disfl])
    n_dis_fl_perc = round(n_dis_fl/n_fl*100, 2)
    
    #FilterKey Statistics
    filter_key_stat = {}
    for filterkey in treelem.FilterKey.unique():
        mask_fk = (treelem.CONTAINERTYPE == 3) & (treelem.FilterKey == filterkey)
        if pd.isnull(filterkey):
            filterkey = 'Undefined'
        filter_key_stat[filterkey] = len(treelem[mask_fk])
        
    #Names Statistics
    names = treelem.loc[treelem.CONTAINERTYPE == 4, 'NAME'].value_counts()
    
    #Points with Thresholds
    n_alarms = len(treelem[(treelem.CONTAINERTYPE == 4) & (~pd.isnull(treelem.SCALARALRMID))])
    mp_w_alarm_perc = round(n_alarms/n_mp*100, 2)
    points_wo_alarms = treelem.loc[(treelem.CONTAINERTYPE == 4) & (pd.isnull(treelem.SCALARALRMID)), ['TREEELEMID']]
    points_wo_alarms.set_index('TREEELEMID', inplace = True)
    

    #DAD statistics
    DAD_types = treelem.loc[treelem.CONTAINERTYPE == 4, ['DADType', 'TREEELEMID']]
    DAD_types.set_index('TREEELEMID', inplace = True)
    DAD_types = DAD_types['DADType'].value_counts()
    print(DAD_types)
    # Need to add mapping to DAD types in order to present not values but Names of the DAD
                            
    
    
    
    
    
    result = {'nodes_statistics': {'mp_number': n_mp,
                                  'assets_number': n_assets,
                                  'fl_number': n_fl},
             'disabled_nodes': {'diabled_mp': {'n': n_dis_mp,
                                              'perc': n_dis_mp_perc},
                               'disabled_assets': {'n': n_dis_assets,
                                                  'perc': n_dis_assets_perc},
                               'disabled_fl': {'n': n_dis_fl,
                                              'perc': n_dis_fl_perc}},
             'filter_key_stat': filter_key_stat,
             'names_stat': names,
             'alarms': {'points_w_alarms': {'n': n_alarms, 
                                            'perc': mp_w_alarm_perc},
                       'points_wo_alarms': points_wo_alarms.to_dict()},
             'DAD': DAD_types}
    
    return result
    

def create_path(node_id = 1, 
               treelem = pd.DataFrame(), 
               logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the input
    if not validate_treelems(treelem, logger):
        return None
    
    level = treelem.loc[treelem.TREEELEMID == node_id, 'BRANCHLEVEL'].item()
    node_name = treelem.loc[treelem.TREEELEMID == node_id, 'NAME'].item()
    path = str(node_name)
    while level > treelem.BRANCHLEVEL.min()+1:
        parent_id = treelem.loc[treelem.TREEELEMID == node_id, 'PARENTID'].item()
        parent_name = treelem.loc[treelem.TREEELEMID == parent_id, 'NAME'].item()
        level = treelem.loc[treelem.TREEELEMID == parent_id, 'BRANCHLEVEL'].item()
        print(level)
        node_id = parent_id
        path = parent_name + "/" + path
    return path
    
    
    
    
def check_type_enveleope(treelem = pd.DataFrame(),
                   logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
       
    # Validation of the input
    if not validate_treelems(treelem, logger):
        return None
    
    #Retrieving Points 
    points = treelem[(treelem.CONTAINERTYPE == 4) & ~treelem.NAME.isin(['MA SIT', 'MI SIT'])]
    
    units_maping = {'in/s': 'velocity',
                    'mm/s': 'velocity',
                    'g': 'acceleration',
                    'gE': 'envelope',
                    'RPM': 'speed',
                    'Hz': 'speed',
                    'F': 'temp',
                    'C': 'temp'}
    regex_types = {
        'velocity': '(^\w*)?( |^)\d*.*V( |$)',
        'temp': '(^\w*)?( |^)\d*.*T( |$)',
        'speed': '(^\w*)?( |^)\d*.*S( |$)',
        'envelope': '(^\w*)?( |^)\d*.*((E1)|(E2)|(E3)|(E4))( |$)',
        'acceleration': '(^\w*)?( |^)\d*.*A( |$)'
    }
    points['meas_type'] = [units_maping[x] if x in units_maping.keys() else 'undefined' for x in points.PointUnitType]
    types_in_treelem = list(set(points.meas_type))
    print(types_in_treelem)
    settings_prob = {'TREEELEMID': [],
                     'NAME': [],
                     'Type': [],
                     'Envelope': [],
                     'Path': []}
    for point_type in types_in_treelem:
        print(point_type)
        if point_type == 'undefined':
            continue
        point_names = points.loc[points.meas_type == point_type, ['NAME', 'TREEELEMID', 'FilterEnvelope', 'PointUnitType', 'Path']]
        regex = regex_types[point_type]
        r_point_type = re.compile(regex)
        bad_meastype = point_names[~point_names.NAME.str.contains(regex)]
        if len(bad_meastype) > 0:
            log.warning(f'Following points has discrepancies between settings and name: {[bad_meastype.TREEELEMID]}')
            settings_prob['TREEELEMID'] = settings_prob['TREEELEMID'] + list(bad_meastype.TREEELEMID)
            settings_prob['NAME'] = settings_prob['NAME'] + list(bad_meastype.NAME)
            settings_prob['Type'] = settings_prob['Type'] + list(bad_meastype.PointUnitType)
            settings_prob['Envelope'] = settings_prob['Envelope'] + len(bad_meastype.PointUnitType)*[np.NaN]
            settings_prob['Path'] = settings_prob['Path'] + list(bad_meastype.Path)
        if point_type == 'envelope':
            point_names.FilterEnvelope = ['E'+str(int(x) - 20599) if x in [20600, 20601, 20602, 20603] else 'Undefined Filter in DB' for x in point_names.FilterEnvelope]
            bad_filter = point_names[[y not in x for x,y in zip(point_names.NAME, point_names.FilterEnvelope)]]
            if len(bad_filter) > 0:
                log.warning(f'Following points has wrong envelope filter: {[bad_filter.TREEELEMID]}')
                settings_prob['TREEELEMID'] = settings_prob['TREEELEMID'] + list(bad_filter.TREEELEMID)
                settings_prob['NAME'] = settings_prob['NAME'] + list(bad_filter.NAME)
                settings_prob['Type'] = settings_prob['Type'] + len(bad_filter.FilterEnvelope)*[np.NaN]
                settings_prob['Envelope'] = settings_prob['Envelope'] + list(bad_filter.FilterEnvelope)
                settings_prob['Path'] = settings_prob['Path'] + list(bad_filter.Path)
    
    #settings_prob['Path'] = [create_path(node_id = x, treelem = treelem) for x in settings_prob['TREEELEMID']]
    
    return settings_prob

def define_path(data):
    path = [[x] for x in data[:, 2]]
    levels = np.unique(data[:, 4])
    data = np.insert(data, 5, '', axis =1)
    for level in levels:
        if level == 0:
            continue
        tmp_df = data[(data[:,4] == level)]
        for element in np.unique(tmp_df[:, 0]):
            parent_id = data[data[:, 0] == element, 1]
            parent_path = data[data[:, 0] == parent_id, 5]
            element_name = data[data[:, 0] == element, 2]
            data[data[:, 0] == element, 5] = parent_path + "/" + element_name
    data = pd.DataFrame(data)
    data.columns = ['TREEELEMID', 'PARENTID', 'NAME', 'CONTAINERTYPE', 'BRANCHLEVEL', 'Path']
    return data

def check_duplications(treelem = pd.DataFrame(), 
                       logger = ''):
    
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the imput.
    if not validate_treelems(treelem, logger):
        return None
    
    #Duplications problems
    log.info(f'Checking hierarchy for ducplicated names in the same FL')
    duplication_problems = {'dupl_in_fl': {},
                            'dupl_in_assets': {}}
    fls_id = set(treelem.loc[treelem.CONTAINERTYPE == 3, 'PARENTID'])
    fls = treelem.loc[treelem.TREEELEMID.isin(list(fls_id)), ['NAME', 'TREEELEMID']]
    for fl_id in fls.TREEELEMID:
        mp_in_fl = []
        assets = treelem.loc[treelem.PARENTID == fl_id, 'TREEELEMID']
        for asset in assets:
            mp_in_asset = list(treelem.loc[treelem.PARENTID == asset, 'NAME'])
            mp_in_fl = mp_in_fl + mp_in_asset
            #Checking for duplications in asset
            first_in_asset = True
            for name in set(mp_in_asset):
                name_count = mp_in_asset.count(name)
                if (name_count > 1) & first_in_asset:
                    duplication_problems['dupl_in_assets'][asset] = []
                    first_in_asset = False
                
                if name_count > 1:
                    duplication_problems['dupl_in_assets'][asset].append(name)
                    log.warning(f'Asset {asset} contains points with duplicated names: List of duplicated names are: {", ".join(duplication_problems["dupl_in_assets"][asset])}')
        
        #Counting duplications for FL
        first_in_fl = True
        for name in set(mp_in_fl):
            name_count = mp_in_fl.count(name)
            if (name_count > 1) & first_in_fl:
                duplication_problems['dupl_in_fl'][fl_id] = []
                first_in_fl = False
            if name_count > 1:
                duplication_problems['dupl_in_fl'][fl_id].append(name)
                log.warning(f'FL {fl_id} contains points with duplicated names: List of duplicated names are: {", ".join(duplication_problems["dupl_in_fl"][fl_id])}')
    
    #Generating common table with issues
    resulted_table = pd.DataFrame()
    for fl_problem_id in duplication_problems['dupl_in_fl'].keys():
        tmp_row = treelem.loc[treelem.TREEELEMID == fl_problem_id, ['TREEELEMID', 'Path']]
        tmp_row['Problem'] = f'FL has points with duplicated names: {duplication_problems["dupl_in_fl"][fl_problem_id]}'
        resulted_table = pd.concat([resulted_table, tmp_row])
    for asset_problem_id in duplication_problems['dupl_in_assets'].keys():
        tmp_row = treelem.loc[treelem.TREEELEMID == asset_problem_id, ['TREEELEMID', 'Path']]
        tmp_row['Problem'] = f'FL has points with duplicated names: {duplication_problems["dupl_in_assets"][asset_problem_id]}'
        resulted_table = pd.concat([resulted_table, tmp_row])    
    resulted_table.reset_index(drop = True, inplace = True)
    
    return resulted_table

def check_hierarchy(treelem = pd.DataFrame(), 
                       logger = ''):
    
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the imput.
    if not validate_treelems(treelem, logger):
        return None
    # Checking that the hierarchy has at least certain amount of layers
    wrong_hier_mask = (treelem.CONTAINERTYPE == 3) & (treelem.BRANCHLEVEL <= 1)
    df_wrong = treelem.loc[wrong_hier_mask, ['TREEELEMID', 'Path']]
    df_wrong['Problem'] = 'Too short hierarchy'
    
    return df_wrong

def check_sequence(treelem = pd.DataFrame(), 
                       logger = ''):
    
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the imput.
    if not validate_treelems(treelem, logger):
        return None
    
    fls_id = set(treelem.loc[treelem.CONTAINERTYPE == 3, 'PARENTID'])
    #Check if we have wrong hierarchy than we can have fl_id which is the same as hierarchy ID
    if treelem.loc[treelem.BRANCHLEVEL == 0, 'TREEELEMID'].item() in list(fls_id):
        fls_id.remove(treelem.loc[treelem.BRANCHLEVEL == 0, 'TREEELEMID'].item())
    fls = treelem.loc[treelem.TREEELEMID.isin(list(fls_id)), ['NAME', 'TREEELEMID']]
    mp_in_fl = {}
    for fl_id in list(fls['TREEELEMID']):
        mp_in_fl[fl_id] = []
        asset_ids = list(treelem.loc[treelem.PARENTID == fl_id, 'TREEELEMID'])
        for asset_id in asset_ids:
            mp_names = list(treelem.loc[treelem.PARENTID == asset_id, 'NAME'])
            mp_in_fl[fl_id] = mp_in_fl[fl_id] + mp_names
        log.info(f'FL with id {fl_id} has {len(mp_in_fl[fl_id])} measurement points with following names: {mp_in_fl[fl_id]}')
    
    
    sequence_problems = {}        
    for fl_id in mp_in_fl.keys():
        sequence_in_fl = []
        for name in set(mp_in_fl[fl_id]):
            try:
                number = re.search('[1-9]{1,3}', name).group(0)
                sequence_in_fl.append(number)
            except AttributeError:
                pass
        sequence_in_fl = list(set(sequence_in_fl))
        sequence_in_fl = [int(x) for x in sequence_in_fl]
        meas_location_count = max(sequence_in_fl)
        meas_location_theory = set(range(1, int(meas_location_count) + 1))
        missing_meas_location = list(set(meas_location_theory) - set(sequence_in_fl))
        if len(missing_meas_location) > 0:
            log.warning(f'We have a missing measurement location in the FL {fl_id}. Missing measurement locations are: {missing_meas_location}')
            sequence_problems[fl_id] = missing_meas_location
    
    resulted_table = pd.DataFrame()
    for id_wrong_seq in sequence_problems.keys():
        df_wrong = treelem.loc[treelem.TREEELEMID == id_wrong_seq, ['TREEELEMID', 'Path']]
        df_wrong['Problem'] = f'Locations {", ".join([str(x) for x in sequence_problems[id_wrong_seq]])} is/are missing in FL' 
        resulted_table = pd.concat([resulted_table, df_wrong])
    resulted_table.reset_index(drop = True, inplace = True)
    return resulted_table

def check_motors(treelem = pd.DataFrame(), 
                       logger = ''):
    
    # Setting logger
    log = logging.getLogger(logger)
    
    # Validation of the imput.
    if not validate_treelems(treelem, logger):
        return None
    motors_mask = (treelem.CONTAINERTYPE == 3) & (treelem.FilterKey.isin(['*Motor']))
    motors = treelem.loc[motors_mask, ['TREEELEMID', 'Path']]
    seq_in_motor = []
    resulted_table = pd.DataFrame()
    for motor_id in motors.TREEELEMID:
        motors_mps = treelem.loc[treelem.PARENTID == motor_id, ['TREEELEMID', 'NAME']]
        for name in set(motors_mps['NAME']):
            try:
                number = re.search('[1-9]{1,3}', name).group(0)
                seq_in_motor.append(number)
            except AttributeError:
                pass
        seq_in_motor = [int(x) for x in seq_in_motor]
        seq_in_motor = list(set(seq_in_motor))
        print(seq_in_motor)
        if len(seq_in_motor) != 0:
            if max(seq_in_motor) > 2:
                df_wrong = motors[motors.TREEELEMID == motor_id]
                wrong_ml = [set(seq_in_motor) - set([1,2])]
                wrong_ml = [str(x) for x in wrong_ml]
                df_wrong['Problem'] = f'Motor has more than 2 locations for MP(s): {", ".join(wrong_ml)}'                        
                resulted_table = pd.concat([resulted_table, df_wrong])
            if max(seq_in_motor) == 1:
                df_wrong = motors[motors.TREEELEMID == motor_id]
                df_wrong['Problem'] = f'Motor has less than 2 measurement locations'                        
                resulted_table = pd.concat([resulted_table, df_wrong])
        else:
            df_wrong = motors[motors.TREEELEMID == motor_id]
            df_wrong['Problem'] = f'Motor has no measurement locations or impossible to detect locations based on names'                        
            resulted_table = pd.concat([resulted_table, df_wrong])
                                               
    return resulted_table
    
def check_thresholds(treelem = pd.DataFrame(), 
                    logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    #Validationof the input
    if not validate_treelems(treelem, logger):
        return None
    
    #Creatnig list of points without thresholds
    points_wo_alarms = treelem.loc[(treelem.CONTAINERTYPE == 4) & 
                               treelem.SCALARALRMID.isna() &  
                               ~treelem.NAME.isin(['MA SIT', 'MI SIT']), ['TREEELEMID','NAME', 'Path', 
                               'ENABLEDANGERLO', 'ENABLEALERTLO', 'ENABLEALERTHI', 'ENABLEDANGERHI',
                               'DANGERLO', 'ALERTLO', 'ALERTHI', 'DANGERHI']]

    #Checking if the thresholds are in correct sequence
    tmp_df = treelem[~treelem.SCALARALRMID.isna()]
    tmp_df.reset_index(drop = True, inplace = True)
    
    results_thresh = {'wrong_alarms': []}
    #Analysing 
    for i in range(len(tmp_df)):
        alarms = list(tmp_df.loc[i, ['DANGERLO', 'ALERTLO', 'ALERTHI', 'DANGERHI']])
        enabled = list(tmp_df.loc[i, ['ENABLEDANGERLO', 'ENABLEALERTLO', 'ENABLEALERTHI', 'ENABLEDANGERHI']])
        enabled_alarms = [x for y, x in enumerate(alarms) if enabled[y] != 0]
        good_alarms = all(enabled_alarms[i] < enabled_alarms[i+1] for i in range(len(enabled_alarms) - 1))
        if not good_alarms:
            results_thresh['wrong_alarms'].append(tmp_df.loc[i, 'TREEELEMID'].item())
            log.warning(f'MP with ID {tmp_df.loc[i, "TREEELEMID"].item()} has a wrong thresholds set.')

    wrong_alarms = treelem.loc[treelem.TREEELEMID.isin(results_thresh['wrong_alarms']), ['TREEELEMID', 'NAME', 'Path']]   
    return {'threshold_issues': wrong_alarms,
            'points_wo_alarms': points_wo_alarms}

def suggest_name(name = '', logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    #Check if we have manual entry RPM/Hz points
    reg_MERPM = re.compile('^manual {1,}entry {1,}\(?((rpm)|(hz))\)?', re.IGNORECASE)
    suggest_name = np.NaN
    if reg_MERPM.search(name):
        #Need to understand if we have Hz or RPM
        regRPM = re.compile('rpm', re.IGNORECASE)
        regHZ = re.compile('hz', re.IGNORECASE)
        if regRPM.search(name):
            suggested_name = '01S Manual Entry RPM'
        if regHZ.search(name):
            suggested_name = '01S Manual Entry Hz'
    return suggested_name
    
    #Device identification
    try:
        device = re.search('^\w{2} ', name).group(0)
        device = device.strip()
    except AttributeError:
        device = ''
    #Checking that device is correct, according to list
    if (device == '') or (device in ['MA', 'MI', 'ME', 'OS', 'TO', 'DV', 'OI']):
        pass
    else:
        #Check for mostly used device in the customer. Suggest this device.
        device = ''
        #If there is no inforamtion about mostly used devices suggest fist with smallest Levenstein distance
        
        
    # Checking nuber
    try:
        location = int(re.search('[0-9]{1,3}', name).group(0))
        print(location)
    except AttributeError:
        location = np.NaN
    
    if ~pd.isna(location):
        if location <= 99:
            location = format(location, '02d')
        else: location = np.NaN
    
    #Checking orientation
    try:
        orientation = re.search('( |^)[0-9]{1,3} ?(A|H|V|R)', name).group(0)
        orientation = orientation[-1]
    except AttributeError:
        orientation = ''
    
    #Checking type of measurements
    try:
        m_type = re.search('(^\w*)?( |^)\d*(A|H|V|R)(A|T|V|S|B|P|G|D|(E1)|(E2)|(E3)|(E4))', name).group(0)
        m_type = m_type.split(' ')[-1]
        m_type = re.search('(A|T|V|S|B|P|G|D|(E1)|(E2)|(E3)|(E4))$', m_type).group(0)
    except AttributeError:
        m_type = ''
    
    # Checking DE NDE
    try:
        de_nde = re.search('(DE)|(NDE)', name).group(0)
    except AttributeError:
        de_nde = ''
    
    print((f'Name {name}', location, orientation, m_type, de_nde))    
    #Checking if we have obligatory parameters
    if ~pd.isna(location) and (orientation != '') and (m_type != ''):
        suggested_name = str(device) + ' ' + str(location) + str(orientation) + str(m_type) + ' ' + de_nde
        suggested_name = suggested_name.strip()
    else:
        suggested_name = np.NaN
    
    return suggested_name

def suggest_settings(resulted_table = pd.DataFrame(),
                    logger = ''):
    # Setting logger
    log = logging.getLogger(logger)
    
    regex = {'Location': '[0-9]{1,3}',
            'Orientation': '( |^)[0-9]{2}(A|H|V|R)',
            'Envelope': '(^\w*)?( |^)[0-9]{2}(A|H|V|R)((E1)|(E2)|(E3)|(E4))'}
    
    settings_columns = ['Location', 'Orientation', 'Envelope']
    for setting in settings_columns:
        resulted_table[setting + "_sgst"] = None
        resulted_list= []
        reg = regex[setting]
        for location in resulted_table.loc[~resulted_table[setting].isna(), 'NAME']:
            try:
                location = re.search(reg, location).group(0)
                if setting == 'Location':
                    location = int(location)
                if setting == 'Orientation':
                    location = location[-1]
                if setting == 'Envelope':
                    location = location[-2:]
                resulted_list.append(location)
            except AttributeError:
                resulted_list.append(None)
                
        resulted_table.loc[~resulted_table[setting].isna(), setting + "_sgst"] = resulted_list
            
    return resulted_table

In [3]:
from celery import Celery
from dash.long_callback import CeleryLongCallbackManager

In [6]:
check_names(unique_list)

NameError: name 'unique_list' is not defined

In [7]:
'bf' in 'fonbfb'

True

In [139]:
db_data = treelems[treelems.DADType != 792]
names= list(set(db_data.loc[db_data.CONTAINERTYPE == 4, 'NAME']))
names_issues = check_names(mp_names=names)
points_w_good_names = db_data[db_data.NAME.isin(names_issues['good_names'])]
location = pd.DataFrame(check_location(treelem=points_w_good_names))
orientation = pd.DataFrame(check_orientation(treelem=points_w_good_names))
m_type = pd.DataFrame(check_type_enveleope(treelem=points_w_good_names))
resulted = pd.merge(location, orientation, on = ['TREEELEMID', 'NAME', 'Path'], how  = 'outer')
resulted = pd.merge(resulted, m_type, on = ['TREEELEMID', 'NAME', 'Path'], how  = 'outer')

['envelope', 'acceleration', 'velocity', 'temp']
envelope
acceleration
velocity
temp


C:\Users\krama\AppData\Local\Temp/ipykernel_5816/2977515185.py:388: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['Location'] = locations_set
C:\Users\krama\AppData\Local\Temp/ipykernel_5816/2977515185.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['Orientation'] = orientations_set
C:\Users\krama\AppData\Local\Temp/ipykernel_5816/2977515185.py:553: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [162]:
suggest_settings(resulted)

,TREEELEMID,NAME,Location,Path,Orientation,Type,Envelope,Location_sgst,Orientation_sgst,Envelope_sgst
0,50731,04HV NDE,2,/BOILER/WELLONS ID FAN/WELLONS ID FAN /04HV NDE,NaN,NaN,NaN,4,None,None
1,50760,04HV NDE,2,/BOILER/EPI FD FAN/BOIL EPI FD FAN FAN/04HV NDE,NaN,NaN,NaN,4,None,None
2,50815,04HE2 NDE,2,/PLANER/CHIPPER/PLANER CHIPPER SFT/04HE2 NDE,NaN,NaN,NaN,4,None,None
3,50873,04HE2 NDE,2,/PLANER/BGH MAIN BLR/PLN BGH MAIN BLR FAN/04HE...,NaN,NaN,NaN,4,None,None
4,50906,04AV NDE,2,/PLANER/SHAVINGS BLR/SHAVINGS BLR BLR/04AV NDE,NaN,NaN,NaN,4,None,None
...,...,...,...,...,...,...,...,...,...,...
1004,52503,04HE3 IP NDE,2,/SAWMILL/DEBARKER LNG I/F CHA/DEBARK LN I/F CH...,NaN,NaN,NaN,4,None,None
1005,52534,05HE2 OP DE,1,/SAWMILL/DEBARKER SHT I/F CHA/DEBARK ST I/F CH...,NaN,NaN,NaN,5,None,None
1006,52574,01HV NDE LF,RPM,/SAWMILL/CHOPSAW I/F CHAIN/CH SAW I/F CH MTR/0...,None,NaN,NaN,1,H,None
1007,52631,03AV IP DE,1,/SAWMILL/CHOPSAW E X-DECK/CH SAW E X-DK GBX/03...,Horizontal,NaN,NaN,3,A,None


In [ ]:
x['wrong_names'].append('010HE3 NDE.1')
[suggest_name(y) for y in x['wrong_names']] 

In [3]:
x['wrong_names']

NameError: name 'x' is not defined

In [3]:
treelems = pd.read_csv('C:/Users/krama/Documents/work/SKF/Vibration - Analyst/Scripts/Customers DB validation/USAEO0905.csv')
for_path = np.array(treelems[['TREEELEMID', 'PARENTID', 'NAME', 'CONTAINERTYPE', 'BRANCHLEVEL']])
x = define_path(for_path)
treelems['Path'] = x['Path']

In [5]:
y = treelems['FilterEnvelope'] - 20599

In [8]:
['E' + str(x) if  else x for x in y]

['Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'E2.0',
 'E2.0',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'E3.0',
 'Enan',
 'Enan',
 'E3.0',
 'E3.0',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E3.0',
 'Enan',
 'E3.0',
 'Enan',
 'E3.0',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'E2.0',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'E2.0',
 'Enan',
 'Enan',
 'E2.0',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'Enan',
 'Enan',
 'E2.0',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'E3.0',
 'Enan',
 'E3.0',
 'E3.0',
 'Enan',
 'Enan',
 'E3.0',
 'E2.0',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'Enan',
 'E2.0',
 'E3.0',
 'Enan',
 'Enan',
 'E2.0',
 'Enan',
 'E3.0',
 'Enan',
 'Enan',
 'E2.0',
 

In [130]:
z = pd.merge(pd.DataFrame(x), pd.DataFrame(y), on=['TREEELEMID', 'NAME', 'Path'], how = 'outer')
f = pd.merge(z, pd.DataFrame(t), on=['TREEELEMID', 'NAME', 'Path'], how = 'outer')

In [131]:
columns = [
    {"name": ["", "TREEELEMID"], "id": "TREEELEMID"},
    {"name": ["", "NAME"], "id": "NAME"},
    {"name": ["Current settings", "Location"], "id": "Location"},
    {"name": ["Current settings", "Orientation"], "id": "Orientation"},
    {"name": ["Current settings", "Type"], "id": "Type"},
    {"name": ["Current settings", "Envelope"], "id": "Envelope"},
    {"name": ["Suggested settings", "Location"], "id": "Location_sgst"},
    {"name": ["Suggested settings", "Orientation"], "id": "Orientation_sgst"},
    {"name": ["Suggested settings", "Type"], "id": "Type_sgst"},
    {"name": ["Suggested settings", "Envelope"], "id": "Envelope_sgst"},
    {"name": ["", "Path"], "id": "Path"},
]

,TREEELEMID,NAME,Type,Envelope,Path,Orientation,Location
0,72548,Manual Entry RPM,RPM,NaN,/B & R/AIR COMPRESSOR=A1/MTR__10026795/Manual ...,None,NaN
1,72575,Manual Entry RPM,RPM,NaN,/B & R/AIR COMPRESSOR=A2/MTR__10026920/Manual ...,None,NaN
2,72616,Manual Entry RPM,RPM,NaN,/B & R/AIR COMPRESSOR=A3/MTR__20013012/Manual ...,None,NaN
3,73079,Manual Entry RPM,RPM,NaN,/B & R/BOOSTER COMP=B8/MTR__20021474/Manual En...,None,NaN
4,72657,Manual Entry RPM,RPM,NaN,/B & R/AIR COMPRESSOR=A4/MTR__20013029/Manual ...,None,NaN
...,...,...,...,...,...,...,...
1775,80149,03HE3 NDE,NaN,NaN,/NEW MACHINES/FF2 Processing/WASHER DRM FIN SC...,NaN,1
1776,80178,03AV NDE,NaN,NaN,/NEW MACHINES/FF2 Processing/TOMRA POTATO SORT...,NaN,1
1777,80179,04HA DE,NaN,NaN,/NEW MACHINES/FF2 Processing/TOMRA POTATO SORT...,NaN,2
1778,80258,05VV DE,NaN,NaN,/NEW MACHINES/SP1 Processing/SL 2/3 BLANCH BLW...,NaN,3


In [132]:
for i in  $(pip list --outdated --format=columns |tail -n +3|cut -d" " -f1); do pip install $i --upgrade; done

True

In [85]:
x = define_path(for_path)

In [83]:
x

,TREEELEMID,PARENTID,NAME,CONTAINERTYPE,BRANCHLEVEL,PATH
0,72562,72558,03VA DE,4,4,/B & R/AIR COMPRESSOR=A1/CPR__10026795/03VA DE
1,72589,72573,CPR__10026920,3,3,/B & R/AIR COMPRESSOR=A2/CPR__10026920
2,72646,72630,05HV DE,4,4,/B & R/AIR COMPRESSOR=A3/CPR__20013012/05HV DE
3,72661,72656,01HV NDE,4,4,/B & R/AIR COMPRESSOR=A4/MTR__20013029/01HV NDE
4,72709,72697,02HV DE,4,4,/B & R/AIR COMPRESSOR=A5/MTR__10009700/02HV DE
...,...,...,...,...,...,...
3458,2700971,74091,MI SIT,4,5,/FF1 PROCESSING/FF1 DRYER/FF1 DRYER FAN_A-UC*/...
3459,2700974,74117,MI SIT,4,5,/FF1 PROCESSING/FF1 DRYER/FF1 DRYER FAN_A-UE/M...
3460,2701031,74611,MI SIT,4,5,/FF2 PROCESSING/FF2 DRYER/FF2 DRYER FAN_B-LW*/...
3461,2701046,74756,MI SIT,4,4,/FF2 PROCESSING/FF2 FRY ERS CIR PMP/MTR__10007...


In [63]:
x = check_type_enveleope(treelem=treelems)
y = check_location(treelem=treelems)
t = check_orientation(treelem=treelems)
z = pd.concat([pd.DataFrame(x),pd.DataFrame(y), pd.DataFrame(t)], axis = 0)

<ipython-input-36-4d1d3f561f28>:666: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['meas_type'] = [units_maping[x] if x in units_maping.keys() else 'undefined' for x in points.PointUnitType]
C:\Users\krama\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
Following points has wrong envelope filter: [754       74928
1003      74993
1010      75118
1087      74977
1165      75039
         ...   
3353      80245
3406      80154
3411      80210
3416    1280294
3438      80190
Name: TREEELEMID, Length: 131, dtype: int64]
Following points has discrepancies between settings and name: [17      72548
40      72575
88 

['acceleration', 'envelope', 'speed', 'velocity']
acceleration
envelope
speed
velocity
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1

<ipython-input-36-4d1d3f561f28>:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['Settings'] = locations_set
<ipython-input-36-4d1d3f561f28>:487: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['Wrong set type'] = 'Orientation'



1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3

1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3


<ipython-input-36-4d1d3f561f28>:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['Settings'] = orientations_set
<ipython-input-36-4d1d3f561f28>:526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['Wrong set type'] = 'Orientation'


2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
4
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
4
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1
3
2
1


In [64]:
z

,TREEELEMID,NAME,Settings,Wrong set type,Path
0,74928,01HE2 NDE,Undefined Filter in DB,Filter type,SL1 PROCESSING/SL1 VAC WTR PMP/MTR__10011099/0...
1,74993,02HE2 DE,Undefined Filter in DB,Filter type,SL1 PROCESSING/SL1 COL SCRN REL/MTR__10011078/...
2,75118,05HE2 IM DE,Undefined Filter in DB,Filter type,SL1 PROCESSING/SL1 CKR DCHRG AGR/GBX__10011081...
3,74977,04HE2 NDE,Undefined Filter in DB,Filter type,SL1 PROCESSING/SL1 COL SCRN PMP/PMP__10011085/...
4,75039,01HE2 NDE,Undefined Filter in DB,Filter type,SL1 PROCESSING/SL1 CKR XFER AGR/MTR__10011082/...
...,...,...,...,...,...
2792,76807,Manual Entry RPM,None,Orientation,PEELER ROOM/PLR 1 DSCHG AUGR/MTR__20027525/Man...
2825,80188,Manual Entry Hz,None,Orientation,NEW MACHINES/SP1 Processing/REYCO PRODUCT BLOW...
2830,80193,01HV NDE LF,None,Orientation,NEW MACHINES/SP1 Processing/REYCO PRODUCT BLOW...
2867,80229,Manual Entry Hz,None,Orientation,NEW MACHINES/SP1 Processing/SL 2/3 BLANCH BLWR...


{'TREEELEMID': None,
 'NAME': None,
 'Settings': None,
 'Wrong set type': None,
 'Path': None}

In [133]:
datetime.now() - datetime.t

datetime.datetime(2022, 2, 4, 15, 59, 40, 304045)

In [39]:
# Table TreelemID that can be used as a testing DF for hierarchy probelms definition
treelems = pd.read_excel('C:/Users/krama/Documents/work/SKF/Vibration - Analyst/Scripts/Customers DB validation/Database Evaluation.xlsx', sheet_name='TREELEMID_v2')
tmp = prepare_hierarchy(treelem=treelems)['treeelem']
stat = db_stat(treelem=tmp)
orientation_issues = check_orientation(treelem = tmp)
location_issues = check_location(treelem = tmp)
duplication_issues = check_duplications(treelem = tmp)
sit_issues = check_sit(treelem = tmp)
names_w_issues = check_names(mp_names=list(tmp.loc[tmp.CONTAINERTYPE == 4, 'NAME'].unique()))
names_issues = define_names_problems(names_w_issues['wrong_names'])

0
1
2
870
1108
3
459
1109
1154
1199
1245
4
460
494
528
564
598
632
666
699
735
769
801
834
1110
1131
1155
1176
1200
1222
1246
1267
5
461
478
495
511
529
547
565
581
599
614
633
647
667
684
700
718
736
754
770
786
802
819
835
853
876
898
894
896
1093
1095
1098
1102


Asset with ID 478 completely disabled all mp inside will be marked and counted as disabled.
FL 834 contains points with duplicated names: List of duplicated names are: 03VV
FL 834 contains points with duplicated names: List of duplicated names are: 03VV, 03HE3
FL 834 contains points with duplicated names: List of duplicated names are: 03VV, 03HE3, 03HV
FL 834 contains points with duplicated names: List of duplicated names are: 03VV, 03HE3, 03HV, 01HV
There are 17 assets without defined filter key. The names are: ['P2 FR1 COMB FAN MTR', 'P2 FR1 COMB FAN FAN', 'P2 FR1 DIL FAN 1 MTR', 'P2 FR2 COMB FAN MTR', 'P2 FR2 COMB FAN FAN', 'P2 FR2 DILT FAN MTR', 'P2 FR2 DILT FAN FAN', 'P2 TUNNEL FUR AIR MT', 'P2 TUNNEL FUR AIR FN', '03HE3', '03HE4', '03HE1', '03HE2', '03HA', '03HV', 'POINT TEST', 'NEW MACHINE']
Asset with ID 700 and Filter Key nan has SIT points.
Asset with ID 736 and Filter Key nan has SIT points.
Asset with ID 770 and Filter Key nan has SIT points.
Asset with ID 802 and Filter Ke

In [36]:
gen_json = {**stat, **orientation_issues, **location_issues, **duplication_issues, **sit_issues, **names_issues}

In [42]:
cust_details = pd.read_excel('cust_details.xlsx')
options = []
for cust in cust_details.customer:
    label = cust_details.loc[cust_details.customer == cust, "customer"].item()
    value = cust_details.loc[cust_details.customer == cust, "short_name"].item()
    options.append({'label': label, 'value': value})
print(options)

[{'label': 'Alamos_Gold', 'value': 'CAAEO21'}, {'label': 'Barrette_Chapais', 'value': 'CAAEO32'}, {'label': 'Big River Steel', 'value': 'USAEO0401'}, {'label': 'Cargill Springdale', 'value': 'USAEO0915'}, {'label': 'Cargill_Itapira', 'value': 'BRAEO3718'}, {'label': 'Certainteed_Missassauga', 'value': 'CAAEO04'}, {'label': 'Chantiers_Chibougamau', 'value': 'CAAEO08'}, {'label': 'CompassMineral_SiftoCanada', 'value': 'CAAEO13'}, {'label': 'Criterion BayView', 'value': 'USAEO0907'}, {'label': 'Crown_Packaging', 'value': 'CAAEO11'}, {'label': 'ERB - MG', 'value': 'BRAEO5402'}, {'label': 'Kroger_Bakery', 'value': 'USAEO0934'}, {'label': 'Kroger_IceCream', 'value': 'USAEO0922'}, {'label': 'Magotteaux_Magog', 'value': 'CAAEO19'}, {'label': 'McCain Foods', 'value': 'USAEO0930'}, {'label': 'McCain_GrandFalls', 'value': 'CAAEO16'}, {'label': 'NationalGypsum_NovaScotia', 'value': 'CAAEO22'}, {'label': 'Nestle_USA', 'value': 'USAEO0929'}, {'label': 'OxyChem', 'value': 'USAEO0927'}, {'label': 'Sol

In [37]:
with open('tmp_audit.json', 'w') as f:
    json.dump(gen_json, f)

In [136]:
tmp.PATH[620]

'/Hierarchy/CONVEYOR HYD/COIL CONV HYD PMP 4/CC HYD 4 PMP'

In [25]:
m['children'].append({'name': 'nyt', 'id': 458, 'children': []})

[]